In [2]:
!pip install -q transformers

In [3]:
import numpy as np
import pandas as pd
import re
import textwrap
import torch
import transformers
from torch.optim import AdamW
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore", category=Warning)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

In [5]:
path = '/kaggle/input/horoscope/horoscope.csv'
df = pd.read_csv(path)
df.tail(10)

,date,sign,text
82526,2023-08-31,gemini,Близнецам сегодня лучше не строить новые планы...
82527,2023-08-31,cancer,"Для Раков это информативный день, но им желате..."
82528,2023-08-31,leo,Сегодня звезды советуют Львам относиться внима...
82529,2023-08-31,virgo,Сегодня Девам лучше не ждать полной откровенно...
82530,2023-08-31,libra,Сегодня звезды советуют Весам заняться преодол...
82531,2023-08-31,scorpio,Сегодня у Скорпионов появляется шанс обрести д...
82532,2023-08-31,sagittarius,"Этот день может сделать Стрельцов скрытными, с..."
82533,2023-08-31,capricorn,Сегодня Козерогам стоит проверять все поступаю...
82534,2023-08-31,aquarius,Сегодня звезды рекомендуют Водолеям контролиро...
82535,2023-08-31,pisces,Сегодня звезды советуют Рыбам не уходить от ра...


In [6]:
text = ' '.join(df['text'].values)

In [7]:
len(text)

34384397

In [8]:
tokens = tokenizer.encode(text, add_special_tokens=True) # токенизация
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

6668406


array([37653,   839, 39588,   567,  1054,  4912,  2493,   329,   379,
         297])

In [9]:
# делим на train и test

ttl = len(tokens)//15 
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*ttl: (i+1)*ttl])
    else:
        test.extend(tokens[i*ttl: (i+1)*ttl])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

6668406 5334720 1333680


In [10]:
from transformers import GPT2LMHeadModel


model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [11]:
batch_size = 8
max_len = 128
epochs = 5

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

optimizer = AdamW(model.parameters(), lr = 1e-5)

total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(
                                            optimizer,
                                            num_warmup_steps=500,
                                            num_training_steps=total_steps)

# задаем точность
def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

5209 1302


In [12]:
# готовим тензоры
def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids

In [13]:
# обучаем модель
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                                token_type_ids=None,
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    torch.save(model.state_dict(), f'model{epoch}.pt')
    torch.save(model, f'entire_model{epoch}.pt')

    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():
            loss, logits, _ = model(batch_ids,
                                token_type_ids=None,
                                labels=batch_ids
                                 ).values()

        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/5 : training


  0%|          | 0/5209 [00:00<?, ?it/s]

epoch 1/5 : validation


  0%|          | 0/1302 [00:00<?, ?it/s]

epoch 2/5 : training


...

  0%|          | 0/1302 [00:00<?, ?it/s]

epoch 5/5 : training


  0%|          | 0/5209 [00:00<?, ?it/s]

epoch 5/5 : validation


  0%|          | 0/1302 [00:00<?, ?it/s]

In [15]:
with torch.inference_mode():
    prompt = """Рыбы"""
    prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
    out = model.generate(
        input_ids=prompt,
        max_length=100,
        num_beams=5,
        do_sample=True,
        temperature=1.5,
        top_k=100,
        top_p=0.8,
        no_repeat_ngram_size=2,
        num_return_sequences=10,
        ).cpu().numpy()
    for out_ in out:
        txt = textwrap.fill(tokenizer.decode(out_), 100)
        print('.'.join(txt.split('.')[:-1]) + '.', end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Рыбы могут провести день в гордом одиночестве – это не лучший момент для романтических свиданий и
новых знакомств. Сегодня Овнам не стоит браться за новые дела – они могут быть отложены на долгий
ящик. Лучше займитесь чем-то интересным и полезным, например, общайтесь с единомышленниками –
сегодня вы точно добьетесь успеха. Полезно общаться с друзьями – в этот день вы можете найти новых
друзей и единомышленников, которые вдохновят вас на новые проекты.
------------------
Рыбы сегодня будут в центре внимания, если не сказать – в самой гуще событий. Сегодня Овнам стоит
быть готовыми к тому, что некоторые ситуации будут складываться не в вашу пользу. Так что
ввязывайтесь в авантюры и начинайте что-то новое – только так вы сможете продвинуться по карьерной
лестнице. Также звезды советуют в этот день прислушиваться к своим ощущениям, чтобы не совершить
ошибок. Не стоит откладывать важные дела на завтра – сейчас они могут повлиять на вашу карьеру.
------------------
Рыбы сегодня будут действо

In [16]:
torch.cuda.empty_cache()

**Видим, что в предсказаниях для одних знаков часто встречаются названия других. 
Попробуем решить эту проблему путем упорядочивания датасета не по дате, а по знакам зодиака и обучим модель еще раз.**

In [17]:
df_sorted = df.sort_values(by=['sign'])

In [18]:
df_sorted

,date,sign,text
34486,2011-11-13,aquarius,В этот день не рекомендуется пускать дела на с...
65602,2019-10-20,aquarius,Сегодня комфорт Водолеев зависит от обстановки...
39934,2013-02-09,aquarius,"Все будет отлично, если вы верите в себя и не ..."
57574,2017-12-20,aquarius,Водолеев сегодня могут ждать не только интерес...
78754,2022-10-20,aquarius,"Это удачный день, когда предложения будут стек..."
...,...,...,...
7853,2005-10-16,virgo,Вам нелегко удается общаться с партнером откры...
20969,2008-10-13,virgo,Утром постарайтесь сосредоточиться на собствен...
39389,2012-12-26,virgo,"Чтобы не совершить ошибку, Девам сегодня доста..."
21065,2008-10-21,virgo,"В дневное время интуиция подскажет вам, как пр..."


In [20]:
text_sorted = ' '.join(df['text'].values)

In [21]:
tokens_sorted = tokenizer.encode(text_sorted, add_special_tokens=True)
tokens_sorted = np.array(tokens_sorted)
print(len(tokens_sorted))
tokens_sorted[:10]

6668406


array([37653,   839, 39588,   567,  1054,  4912,  2493,   329,   379,
         297])

In [22]:
l = len(tokens_sorted)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens_sorted[i*l: (i+1)*l])
    else:
        test.extend(tokens_sorted[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens_sorted), len(train), len(test))

6668406 5334720 1333680


In [23]:
model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [24]:
batch_size = 8
max_len = 128
epochs = 5

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

5209 1302


In [25]:
optimizer = AdamW(model.parameters(), lr = 1e-5)

total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(
                                            optimizer,
                                            num_warmup_steps=500,
                                            num_training_steps=total_steps)


def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

In [26]:
def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids

In [27]:
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                                token_type_ids=None,
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    torch.save(model.state_dict(), f'model{epoch}.pt')
    torch.save(model, f'entire_model{epoch}.pt')

    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():
            loss, logits, _ = model(batch_ids,
                                token_type_ids=None,
                                labels=batch_ids
                                 ).values()

        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/5 : training


  0%|          | 0/5209 [00:00<?, ?it/s]

epoch 1/5 : validation


  0%|          | 0/1302 [00:00<?, ?it/s]

epoch 2/5 : training


...

  0%|          | 0/1302 [00:00<?, ?it/s]

epoch 5/5 : training


  0%|          | 0/5209 [00:00<?, ?it/s]

epoch 5/5 : validation


  0%|          | 0/1302 [00:00<?, ?it/s]

In [67]:
with torch.inference_mode():
    prompt = """Близнецы"""
    prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
    out = model.generate(
        input_ids=prompt,
        max_length=80,
        num_beams=5,
        do_sample=True,
        temperature=1.5,
        top_k=80,
        top_p=0.8,
        no_repeat_ngram_size=2,
        num_return_sequences=5,
        ).cpu().numpy()
    for out_ in out:
        txt = textwrap.fill(tokenizer.decode(out_), 100)
        if 'рожден' not in txt: # фильтруем фразы типа "Львам, рожденным в январе", "Львам, рожденным под знаком Льва"
            print('.'.join(txt.split('.')[:-1]) + '.', end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Близнецы в этот день склонны действовать в согласии со своей интуицией. В то же время не стоит
забывать и о своих целях. Если вы не доверяете своим чувствам и интересам, лучше не поддаваться
импульсу и не делать поспешных выводов. Стоит избегать необдуманных поступков, необузданного
эгоизма, чрезмерной чувствительности.
------------------
Близнецы сегодня могут позволить себе больше, чем обычно. Не стоит поддаваться искушению взять что-
то в кредит или вложить средства в сомнительную авантюру. День хорош для отдыха, общения,
романтического свидания, семейных торжеств, помолвки или бракосочетания, а также для семейных
вечеров, приема гостей.
------------------
Близнецы могут столкнуться с непониманием со стороны других людей из-за их холодности или
подозрительности. Вы можете быть не согласны с тем, что происходит в доме, семье, в отношениях с
родителями, домочадцами, а также с мнением друзей о вас. Это не лучший момент для выяснения
отношений, особенно с теми, кого вы любите или уважае